In [26]:
import sqlalchemy
import pandas as pd

In [138]:
from sqlalchemy_utils.functions import database_exists, create_database

# Connecting to AWS Database

In [111]:
# Defining key parameters for connection to database
host="sqlalchemytutorial.******.eu-central-1.rds.amazonaws.com"
port=3306
dbname="TUTORIAL"
user="admin"
password="Baloney1" #now they make you add numbers...

In [24]:
# Connecting to database
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'
            .format(user, password,host, dbname)).connect()

# Preparing csv data for Relational Database

In [119]:
# Reading data to be saved in DB
data = pd.read_csv(r"https://raw.githubusercontent.com/Jan-Majewski/Medium_articles/master/02_AWS_RDS_with_SQLAlchemy/London_property_data.csv")

In [140]:
data=pd.read_csv('London_property_data.csv')

In [142]:
# Making sure that id is int
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1035 non-null   int64  
 1   MonthlyPrice   1034 non-null   float64
 2   Currency       1035 non-null   object 
 3   bedrooms       486 non-null    float64
 4   bathrooms      788 non-null    float64
 5   latitude       1035 non-null   float64
 6   longitude      1035 non-null   float64
 7   download_time  1035 non-null   object 
 8   City           1035 non-null   object 
 9   Country        1035 non-null   object 
dtypes: float64(5), int64(1), object(4)
memory usage: 81.0+ KB


In [144]:
data.columns

Index(['id', 'MonthlyPrice', 'Currency', 'bedrooms', 'bathrooms', 'latitude',
       'longitude', 'download_time', 'City', 'Country'],
      dtype='object')

In [145]:
# Choosing features for each of 3 tables - remember to have id in each to make joining possible
property_features=['id','bedrooms', 'bathrooms']
location_features=['id','latitude','longitude','City', 'Country']
price_features=['id', 'MonthlyPrice','download_time']

In [146]:
table_properties=data[property_features]
table_locations=data[location_features]
table_prices=data[price_features]

In [147]:
table_properties.head()

,id,bedrooms,bathrooms
0,87559256,1.0,1.0
1,72990621,1.0,1.0
2,70875120,3.0,1.0
3,83795935,NaN,1.0
4,72973739,NaN,NaN


In [148]:
table_locations.head()

,id,latitude,longitude,City,Country
0,87559256,51.424110,-0.054950,London,UK
1,72990621,51.534100,-0.201950,London,UK
2,70875120,51.525940,-0.162630,London,UK
3,83795935,51.456526,-0.200109,London,UK
4,72973739,51.549730,-0.183310,London,UK


In [149]:
table_prices.head()

,id,MonthlyPrice,download_time
0,87559256,1000.0,2020-10-04 12:09:00.062859
1,72990621,1000.0,2020-10-04 12:09:00.062859
2,70875120,1057.0,2020-10-04 12:09:00.062859
3,83795935,1000.0,2020-10-04 12:09:00.062859
4,72973739,1040.0,2020-10-04 12:09:00.062859


# Writing data to database

In [130]:
# Write data into 3 separate tables
table_property.to_sql(con=database_connection, name="PROPERTIES", if_exists='replace',chunksize=100, index=False)
table_location.to_sql(con=database_connection, name="LOCATIONS", if_exists='replace',chunksize=100, index=False)
table_prices.to_sql(con=database_connection, name="PRICES", if_exists='replace',chunksize=100, index=False)

# Querying data with SQL

In [132]:
# Creating SQL query to use together with database_connection in next line

sql_query='''
SELECT * FROM PRICES a
JOIN LOCATIONS b
ON a.id=b.id
JOIN PROPERTIES c
ON a.id=c.id
'''

In [133]:
property_data=pd.read_sql(sql_query,con=database_connection)

In [134]:
# Queried data in dataframe format
property_data.head()

,id,MonthlyPrice,download_time,id,latitude,longitude,City,Conutry,id,bedrooms,bathrooms
0,87559256,1000.0,2020-10-04 12:09:00.062859,87559256,51.424110,-0.054950,London,UK,87559256,1.0,1.0
1,72990621,1000.0,2020-10-04 12:09:00.062859,72990621,51.534100,-0.201950,London,UK,72990621,1.0,1.0
2,70875120,1057.0,2020-10-04 12:09:00.062859,70875120,51.525940,-0.162630,London,UK,70875120,3.0,1.0
3,83795935,1000.0,2020-10-04 12:09:00.062859,83795935,51.456526,-0.200109,London,UK,83795935,NaN,1.0
4,72973739,1040.0,2020-10-04 12:09:00.062859,72973739,51.549730,-0.183310,London,UK,72973739,NaN,NaN


In [135]:
property_data.shape

(1035, 11)

# Creating new schema

In [22]:
#Creating a new schema for Advanced tutorial with use of sqlalchemy_utils.functions

dbname_2='TUTORIAL_ADVANCED'
database_connection_2 = 'mysql+mysqlconnector://{0}:{1}@{2}/{3}'.format(user, password,host, dbname_2)

if not database_exists(database_connection_2):
    create_database(database_connection_2)